<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/get_ticker_data_20241223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf

# List of tickers
tickers = ["SPY", "TQQQ", "QQQ", "SQQQ", "EEM", "XLF", "GLD", "XLE", "EFA", "GDX", "XLK", "TLT", "XLV", "FXI", "XLY", "XLI", "XLU", "XLP", "XLB", "TSLA", "AMD", "AMZN", "META", "NVDA", "BAC", "F", "LCID", "WFC", "T", "PLTR", "GE", "GOOGL", "GOOG", "PYPL", "NFLX", "MRO", "PFE", "SOFI", "UBER", "XOM", "AAP", "MS", "BMY", "GM", "OXY", "C", "CCL", "SNAP", "SQ", "NIO", "X", "DAL", "FCX", "BABA", "DVN", "MPC", "RIVN", "UAL", "AAL", "GOLD", "ABBV", "HAL", "TGT", "MU", "HOOD", "DKNG", "COP", "CVE", "PENN"]

# Download data since 1995
data = yf.download(tickers, start="2004-01-01", end="2024-12-31", group_by="ticker")

# Save each ticker's data as a CSV
for ticker in tickers:
    data[ticker].to_csv(f"{ticker}_OHLC.csv")

print("OHLC data downloaded and saved.")


In [ ]:
import pandas as pd
import os

# Use same tickers as cell 1
tickers = ["SPY", "TQQQ", "QQQ", "SQQQ", "EEM", "XLF", "GLD", "XLE", "EFA", "GDX", "XLK", "TLT", "XLV", "FXI", "XLY", "XLI", "XLU", "XLP", "XLB", "TSLA", "AMD", "AMZN", "META", "NVDA", "BAC", "F", "LCID", "WFC", "T", "PLTR", "GE", "GOOGL", "GOOG", "PYPL", "NFLX", "MRO", "PFE", "SOFI", "UBER", "XOM", "AAP", "MS", "BMY", "GM", "OXY", "C", "CCL", "SNAP", "SQ", "NIO", "X", "DAL", "FCX", "BABA", "DVN", "MPC", "RIVN", "UAL", "AAL", "GOLD", "ABBV", "HAL", "TGT", "MU", "HOOD", "DKNG", "COP", "CVE", "PENN"]

# Create export directories if they don't exist
os.makedirs('/content/export_daily', exist_ok=True)
os.makedirs('/content/export_weekly', exist_ok=True)
os.makedirs('/content/export_monthly', exist_ok=True)

def validate_data(df, ticker):
    """Validate OHLC data for errors"""
    # Store original row count
    original_count = len(df)

    # Round all values to 4 decimal places
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].round(4)

    # Create mask for each condition
    non_zero = (df['Open'] > 0) & (df['High'] > 0) & (df['Low'] > 0) & (df['Close'] > 0)
    valid_low = (df['Low'] <= df['Close']) & (df['Low'] <= df['Open'])
    valid_high = (df['High'] >= df['Close']) & (df['High'] >= df['Open'])
    valid_range = (df['High'] - df['Low']) > 0

    # Combine all conditions
    valid_rows = non_zero & valid_low & valid_high & valid_range

    # Filter data
    df_clean = df[valid_rows].copy()

    # Report removed rows
    removed_count = original_count - len(df_clean)
    if removed_count > 0:
        print(f"{ticker}: Removed {removed_count} invalid rows out of {original_count}")

    return df_clean

def load_data(ticker):
    file_path = f'{ticker}_OHLC.csv'
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])

    # Ensure column order for daily data
    column_order = ['Date', 'Open', 'High', 'Low', 'Close']
    df = df[column_order]

    # Validate and clean data before proceeding
    df = validate_data(df, ticker)

    return df

def aggregate_data(df):
    agg_rules = {
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }

    weekly_data = df.set_index('Date').resample('W').agg(agg_rules).reset_index()
    monthly_data = df.set_index('Date').resample('M').agg(agg_rules).reset_index()

    column_order = ['Date', 'Open', 'High', 'Low', 'Close']
    weekly_data = weekly_data[column_order]
    monthly_data = monthly_data[column_order]

    # Validate aggregated data
    weekly_data = validate_data(weekly_data, f"{df.name}_weekly")
    monthly_data = validate_data(monthly_data, f"{df.name}_monthly")

    return weekly_data, monthly_data

def save_to_csv(data, ticker, period):
    if period == 'daily':
        file_path = f'/content/export_daily/{ticker}_daily.csv'
    elif period == 'weekly':
        file_path = f'/content/export_weekly/{ticker}_weekly.csv'
    else:
        file_path = f'/content/export_monthly/{ticker}_monthly.csv'

    data.to_csv(file_path, index=False)
    print(f"Saved {file_path}")

def process_ticker(ticker):
    print(f"Processing {ticker}...")
    try:
        # Load and save daily data
        df = load_data(ticker)
        df.name = ticker  # Add name attribute for reference in aggregation

        if len(df) > 0:  # Only proceed if we have valid data
            save_to_csv(df, ticker, 'daily')

            # Process and save weekly/monthly data
            weekly_data, monthly_data = aggregate_data(df)

            if len(weekly_data) > 0:
                save_to_csv(weekly_data, ticker, 'weekly')
            if len(monthly_data) > 0:
                save_to_csv(monthly_data, ticker, 'monthly')

            print(f"Successfully processed {ticker}")
        else:
            print(f"No valid data for {ticker}")

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")

# Process all tickers
for ticker in tickers:
    process_ticker(ticker)

In [ ]:
import pandas as pd
import os

# Use same tickers as cell 1
tickers = ["SPY", "TQQQ", "QQQ", "SQQQ", "EEM", "XLF", "GLD", "XLE", "EFA", "GDX", "XLK",
           "TLT", "XLV", "FXI", "XLY",
           "XLI", "XLU", "XLP", "XLB"]

# Create export directories if they don't exist
os.makedirs('/content/export_daily', exist_ok=True)
os.makedirs('/content/export_weekly', exist_ok=True)
os.makedirs('/content/export_monthly', exist_ok=True)

def load_data(ticker):
    file_path = f'{ticker}_OHLC.csv'
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])

    # Ensure column order for daily data
    column_order = ['Date', 'Open', 'High', 'Low', 'Close']
    df = df[column_order]

    return df

def aggregate_data(df):
    agg_rules = {
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }

    weekly_data = df.set_index('Date').resample('W').agg(agg_rules).reset_index()
    monthly_data = df.set_index('Date').resample('M').agg(agg_rules).reset_index()

    column_order = ['Date', 'Open', 'High', 'Low', 'Close']
    weekly_data = weekly_data[column_order]
    monthly_data = monthly_data[column_order]

    return weekly_data, monthly_data

def save_to_csv(data, ticker, period):
    if period == 'daily':
        file_path = f'/content/export_daily/{ticker}_daily.csv'
    elif period == 'weekly':
        file_path = f'/content/export_weekly/{ticker}_weekly.csv'
    else:
        file_path = f'/content/export_monthly/{ticker}_monthly.csv'

    data.to_csv(file_path, index=False)
    print(f"Saved {file_path}")

def process_ticker(ticker):
    print(f"Processing {ticker}...")
    try:
        # Load and save daily data
        df = load_data(ticker)
        save_to_csv(df, ticker, 'daily')

        # Process and save weekly/monthly data
        weekly_data, monthly_data = aggregate_data(df)
        save_to_csv(weekly_data, ticker, 'weekly')
        save_to_csv(monthly_data, ticker, 'monthly')

        print(f"Successfully processed {ticker}")
    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")

# Process all tickers
for ticker in tickers:
    process_ticker(ticker)

In [ ]:
from google.colab import files
import os

def count_and_download_files(directory, period):
    file_count = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
    print(f"Found {file_count} {period} files")

    # Download files
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.download(filepath)

print("\nCounting and downloading files...\n")
count_and_download_files('/content/export_monthly', 'monthly')

In [ ]:
from google.colab import files
import os
import zipfile

def zip_and_download(source_dir, period):
    # Count files
    file_count = len([name for name in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, name))])
    print(f"Found {file_count} {period} files")

    # Create zip file
    zip_filename = f'{period}_files.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for file in os.listdir(source_dir):
            file_path = os.path.join(source_dir, file)
            if os.path.isfile(file_path):
                zipf.write(file_path, arcname=file)

    # Download zip file
    files.download(zip_filename)
    print(f"Downloaded {zip_filename}")

# Create and download zips for each period
print("\nCreating and downloading zip files...\n")
zip_and_download('/content/export_monthly', 'monthly')
zip_and_download('/content/export_weekly', 'weekly')
#zip_and_download('/content/export_daily', 'daily')

In [ ]:
from google.colab import files
import os

def count_and_download_files(directory, period):
    file_count = len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
    print(f"Found {file_count} {period} files")

    # Download files
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            files.download(filepath)

print("\nCounting and downloading files...\n")
count_and_download_files('/content/export_daily', 'daily')
count_and_download_files('/content/export_weekly', 'weekly')
count_and_download_files('/content/export_monthly', 'monthly')

In [ ]:
import pandas as pd

# Load data from CSV
def load_data(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    # Set 'Date' as the index
    df.set_index('Date', inplace=True)
    return df

# Aggregate data to weekly and monthly levels
def aggregate_data(df):
    # Weekly aggregation (sum)
    weekly_data = df.resample('W').sum()

    # Monthly aggregation (mean)
    monthly_data = df.resample('ME').mean()

    return weekly_data, monthly_data

# Save aggregated data to CSV
def save_to_csv(data, file_name):
    data.to_csv(file_name)
    print(f"Saved {file_name}")

# Main program
if __name__ == "__main__":
    # Path to the input CSV file
    input_file = "/content/"  # Replace with your file path

    print("Loading data from CSV...")
    df = load_data(input_file)
    print("Daily Data:")
    print(df.head())

    print("\nAggregating data to weekly and monthly levels...")
    weekly_data, monthly_data = aggregate_data(df)

    print("\nWeekly Data (Sum):")
    print(weekly_data.head())

    print("\nMonthly Data (Mean):")
    print(monthly_data.head())

    # Save aggregated data to CSV files
    save_to_csv(weekly_data, "XLB_weekly_data.csv")
    save_to_csv(monthly_data, "XLB_monthly_data.csv")
    print("\nAggregated data saved to CSV files.")
